In [1]:
import json
import os
from datasets import Dataset, DatasetDict
from transformers import PreTrainedTokenizerFast
import numpy as np

2024-05-29 14:20:49.173427: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-29 14:20:49.367200: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.4/lib64:/usr/local/cuda-11.4/lib64::/usr/local/cuda/lib64:/usr/local/cuda/lib64
2024-05-29 14:20:49.367222: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2024-05-29 14:20:49.395346: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for pl

In [5]:
# Open the JSON file for reading
with open('data_0718_re_2.json', 'r') as f:
    # Load the JSON data from the file
    data = json.load(f)

In [6]:
len(data)

10475194

In [7]:
import random

# Example list of data
#data = ["example_1", "example_2", "example_3", "example_4", "example_5", "example_6", "example_7", "example_8", "example_9", "example_10"]

# Shuffle the data
random.shuffle(data)

# Split the data into train, validation, and test sets
num_examples = len(data)
num_train = int(num_examples * 0.7)  # 70% for training
num_val = int(num_examples * 0.2)  # 20% for validation
num_test = num_examples - num_train - num_val  # rest for test

train_data = data[:num_train]
val_data = data[num_train:num_train+num_val]
test_data = data[num_train+num_val:]

# Save train, validation, and test sets to text files
with open("0529_text_data/train.txt", "w") as f:
    for item in train_data:
        f.write("%s\n" % item)

with open("0529_text_data/val.txt", "w") as f:
    for item in val_data:
        f.write("%s\n" % item)

with open("0529_text_data/test.txt", "w") as f:
    for item in test_data:
        f.write("%s\n" % item)

In [9]:
from datasets import load_dataset
# Loading the dataset
text_datasets = {
    "train": ['0529_text_data/train.txt'],
    "eval": ['0529_text_data/val.txt'],
    "test": ['0529_text_data/test.txt']
}

dataset = load_dataset("text", data_files=text_datasets, cache_dir="0529_bpe_token")

Computing checksums:  33%|###3      | 1/3 [00:06<00:13,  6.92s/it]

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating eval split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset text downloaded and prepared to /lwork01/AJ/0523_gpt/0529_bpe_token/text/default-ea26d02be51a4bb7/0.0.0/cb1e9bd71a82ad27976be3b12b407850fe2837d80c22c5e03a28949843a8ace2. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [11]:
from tokenizers import Tokenizer

# Load the tokenizer
loaded_tokenizer = Tokenizer.from_file('240523_space_bpe.json')

In [12]:
from tokenizers import Encoding
from typing import List

def pad_encodings(encodings: List[Encoding]):
    max_length = max([len(encoding.ids) for encoding in encodings])
    input_ids = []
    attention_mask = []
    for encoding in encodings:
        input_ids.append(encoding.ids + [0] * (max_length - len(encoding.ids)))
        attention_mask.append(encoding.attention_mask + [0] * (max_length - len(encoding.attention_mask)))
    return {"input_ids": input_ids, "attention_mask": attention_mask}

def encode_batch(batch):
    encodings = loaded_tokenizer.encode_batch(batch["text"])
    return pad_encodings(encodings)

tokenized_dataset = dataset.map(
    encode_batch,
    batched=True,
    num_proc=4,  # Increase if your machine has more cores
    remove_columns=["text"],
)

#3:   0%|          | 0/1834 [00:00<?, ?ba/s]

#0:   0%|          | 0/1834 [00:00<?, ?ba/s]

#1:   0%|          | 0/1834 [00:00<?, ?ba/s]

#2:   0%|          | 0/1834 [00:00<?, ?ba/s]

#0:   0%|          | 0/524 [00:00<?, ?ba/s]

...

#2:   0%|          | 0/524 [00:00<?, ?ba/s]

#0:   0%|          | 0/262 [00:00<?, ?ba/s]

#2:   0%|          | 0/262 [00:00<?, ?ba/s]

#1:   0%|          | 0/262 [00:00<?, ?ba/s]

#3:   0%|          | 0/262 [00:00<?, ?ba/s]

In [13]:
# Saving the tokenized dataset
tokenized_dataset.save_to_disk('0529_tokenized_dataset')

Saving the dataset (0/8 shards):   0%|          | 0/7332635 [00:00<?, ? examples/s]

Saving the dataset (0/3 shards):   0%|          | 0/2095038 [00:00<?, ? examples/s]

Saving the dataset (0/2 shards):   0%|          | 0/1047521 [00:00<?, ? examples/s]

## gpt2 training

In [ ]:
from glob import glob
import os
import pandas as pd
from tqdm import tqdm
import json
import copy
import re
import datasets
from datasets import load_dataset
from tokenizers import ByteLevelBPETokenizer
from transformers import GPT2TokenizerFast
from transformers import GPT2Tokenizer
from datasets import load_dataset
import torch
from datasets import load_from_disk
from transformers import GPT2LMHeadModel, GPT2Config, Trainer, TrainingArguments

In [15]:
from transformers import PreTrainedTokenizerFast
from transformers import DataCollatorForLanguageModeling

In [18]:
from transformers import GPT2Config, GPT2LMHeadModel
from transformers import Trainer, TrainingArguments

In [16]:
#tokenized_dataset = load_from_disk('/mnt/prj/AJ/3bmgpt/0523_space_tokenized_dataset')

tokenizer = PreTrainedTokenizerFast(tokenizer_file="240523_space_bpe.json")
tokenizer._tokenizer.enable_padding(length=None)  # set max length to 1024 if desired


if not tokenizer.pad_token:
    tokenizer.add_tokens(["[PAD]"])
    tokenizer.pad_token = "[PAD]"

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=False
)

Using pad_token, but it is not set yet.


In [19]:
# Assuming your tokenized_dataset is a dictionary with "train", "eval" and "test" datasets
# Each of these datasets have the keys "input_ids" and "attention_mask"

# Create a configuration for GPT-2 medium
# config = GPT2Config(
#     vocab_size=len(tokenizer),  # Make sure you define 'tokenizer' earlier in your script
#     n_positions=1024,
#     n_ctx=1024,
#     n_embd=1024,  # Increased from 768 for GPT-2 small
#     n_layer=24,   # Increased from 12 for GPT-2 small
#     n_head=16     # Increased from 12 for GPT-2 small
# )

config = GPT2Config(
    vocab_size=len(tokenizer),  # Ensure 'custom_tokenizer' is defined earlier
    n_positions=1024,
    n_ctx=1024,
    n_embd=768,  # Reduced from 1024 for GPT-2 medium
    n_layer=12,  # Reduced from 24 for GPT-2 medium
    n_head=12   # Reduced from 16 for GPT-2 medium
)


# Instantiate a new GPT-2 model
model = GPT2LMHeadModel(config)

model.tokenizer = tokenizer
model.resize_token_embeddings(len(tokenizer))

Embedding(5051, 768)

In [20]:
# Set training arguments
training_args = TrainingArguments(
    output_dir="0529_gpt2_meidum_space", # The output directory
    overwrite_output_dir=True, # overwrite the content of the output directory
    num_train_epochs=3, # number of training epochs
    per_device_train_batch_size=2, # batch size for training 32
    per_device_eval_batch_size=4,  # batch size for evaluation 64
    max_steps=2810238,
    eval_steps=2000, # Number of update steps between two evaluations. 400
    save_steps=100000, # after # steps model is saved  800
    logging_steps=2000,
    evaluation_strategy='steps',
    learning_rate=1e-4,
    weight_decay=0.01,
    warmup_steps=500, # number of warmup steps for learning rate 
)
print(training_args)

TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=2000,
evaluation_strategy=steps,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
fsdp=[],
fsdp_min_num_params=0,
fsdp_transformer_layer_cls_to_wrap=None,
full_determinism=False,
gradient_accumulation_steps=1,
gradient_checkpointing=False,
greater_is_better=None,
group_by_length=False,
half_precision_backend=auto,
hub_model_id=None,
hub_private_repo=False,
hub_strategy=every_save,
hub_token=<HUB_TOKEN>,
ignore_data_skip=False,
include_inputs_for_metrics=False,
jit_mode_eval=False,
label_n

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="0529_gpt2_medium_space", # The output directory
    overwrite_output_dir=True, # Overwrite the content of the output directory
    num_train_epochs=3, # Number of training epochs
    per_device_train_batch_size=32, # Batch size for training
    per_device_eval_batch_size=64,  # Batch size for evaluation
    max_steps=68728, # Total training steps
    eval_steps=500, # Evaluate every 500 steps
    save_steps=3500, # Save model every 3500 steps
    logging_steps=2000, # Log every 2000 steps
    evaluation_strategy='steps', # Use steps for evaluation strategy
    learning_rate=1e-4, # Learning rate
    weight_decay=0.01, # Weight decay
    warmup_steps=500, # Number of warmup steps for learning rate
)

print(training_args)


In [21]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 7332635
    })
    eval: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 2095038
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 1047521
    })
})

In [ ]:
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=False
)

In [22]:
# Create a Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["eval"],
    data_collator=data_collator
)

max_steps is given, it will override any value given in num_train_epochs


In [25]:
print(len(tokenized_dataset['train']), len(tokenized_dataset["eval"]))

7332635 2095038


In [23]:
# Train the model
trainer.train()

/lwork01/anaconda3/SE3-nvidia/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 7332635
  Num Epochs = 1
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 2
  Gradient Accumulation steps = 1
  Total optimization steps = 2810238
  Number of trainable parameters = 89721600
You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss


***** Running Evaluation *****
  Num examples = 2095038
  Batch size = 4


KeyboardInterrupt: 

In [ ]:
# Save the model
model.save_pretrained("/mnt/prj/AJ/3bmgpt/PLI-GPT2-main/gpt2_meidum_space")

# After the training
log_history = trainer.state.log_history

# Extract the losses from the log history
train_losses = [entry['loss'] for entry in log_history if 'loss' in entry]
eval_losses = [entry['eval_loss'] for entry in log_history if 'eval_loss' in entry]

# Save to a file
with open('train_history.json', 'w') as f:
    json.dump({
        'train_loss': train_losses,
        'eval_loss': eval_losses
    }, f)